<a href="https://colab.research.google.com/github/ruanvirginio/scriptsMestrado/blob/main/tratamentoDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import os
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:.2f}'.format

warnings.filterwarnings("ignore")

url = "https://media.githubusercontent.com/media/ruanvirginio/scriptsMestrado/refs/heads/main/Medicoes_2018-2024.csv"

df = pd.read_csv(url,  sep=',', encoding='latin-1', skiprows=1)

df['Potência Ativa'] = pd.to_numeric(df['Potência Ativa'].str.replace(',', '.'), errors='coerce')
df['Potência Reativa'] = pd.to_numeric(df['Potência Reativa'].str.replace(',', '.'), errors='coerce')

# df.tail(3)


In [ ]:
# Filtrando pra um transformador específico
# df_ctr = df[df['TRAFO'] == 'CTR_DJ_12B1']

# Criar o boxplot para a potência ativa de CTR-02T1
# plt.figure(figsize=(10, 6))
# sns.boxplot(y='P', data=df_ctr)
# plt.title('Boxplot da Potência Ativa para o Transformador CTR-02T1')
# plt.ylabel('Potência Ativa')
# plt.show()

In [ ]:
# Função para aplicar o filtro IQR
def filtrar_coluna_iqr(df, coluna):
    Q1 = df[coluna].quantile(0.25)
    Q3 = df[coluna].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 2 * IQR
    upper_bound = Q3 + 3.2 * IQR

    return df[(df[coluna] >= lower_bound) & (df[coluna] <= upper_bound)]

# Função para aplicar o filtro baseado em Média Móvel
def filtrar_coluna_media_movel(df, coluna, janela=20, threshold=4):

    media_movel = df[coluna].rolling(window=janela, center=True).mean()
    desvio = np.abs(df[coluna] - media_movel)

    limite_superior = media_movel + threshold * desvio.std()
    limite_inferior = media_movel - threshold * desvio.std()

    return df[(df[coluna] >= limite_inferior) & (df[coluna] <= limite_superior)]

# Função para aplicar o filtro baseado em Z-Score
def filtrar_coluna_zscore(df, coluna, threshold=3):
    media = df[coluna].mean()
    desvio_padrao = df[coluna].std()
    z_scores = (df[coluna] - media) / desvio_padrao

    return df[np.abs(z_scores) < threshold]

# Função que aplica o filtro escolhido
def aplicar_filtro(df, coluna, metodo='iqr', janela=20, threshold=4, z_threshold=3):
    if metodo == 'iqr':
        return filtrar_coluna_iqr(df, coluna)
    elif metodo == 'media_movel':
        return filtrar_coluna_media_movel(df, coluna, janela=janela, threshold=threshold)
    elif metodo == 'zscore':
        return filtrar_coluna_zscore(df, coluna, threshold=z_threshold)
    else:
        raise ValueError("Método inválido! Escolha entre 'iqr', 'media_movel' ou 'zscore'.")


In [ ]:
df.rename(columns={
    'Potência Ativa': 'P',
    'Potência Reativa': 'Q',
    'Data/Hora Medição': 'datahora',
    'Equipamento Medição': 'TRAFO'
}, inplace=True)

# Convertendo para datetime e salvando o módulo dos valores, pois pode ocorrer valores negativos
# df['datahora'] = pd.to_datetime(df['datahora'], dayfirst=True, errors='coerce')
df['datahora'] = pd.to_datetime(df['datahora'], format='%d/%m/%Y %H:%M:%S')

df['P'] = df['P'].abs()  # salvando o módulo dos valores
df['Q'] = df['Q'].abs()
df['S'] = np.sqrt(df['P']**2 + df['Q']**2)  # Calculando a potência aparente (S), em kVA

# Removendo linhas com S = 0
df = df[df['S'] != 0]

# DataFrame final para armazenar os dados filtrados
df_filtrado = pd.DataFrame()

# Escolha do método de filtragem ('iqr', 'media_movel' ou 'zscore')
metodo_filtro = 'iqr'

# Aplicando o filtro pra limpar os outliers pra cada transformador
for trafo in df['TRAFO'].unique():
    df_trafo = df[df['TRAFO'] == trafo]

    df_trafo_filtrado = aplicar_filtro(df_trafo, 'S', metodo=metodo_filtro, janela=20, threshold=4, z_threshold=3)

    # Adicionar os dados filtrados ao DataFrame final
    df_filtrado = pd.concat([df_filtrado, df_trafo_filtrado], ignore_index=True)

# Ordenando e removendo duplicatas
df_filtrado = df_filtrado.sort_values(by=['TRAFO', 'datahora'])
df_filtrado = df_filtrado.drop_duplicates(subset=['datahora', 'TRAFO'])

df_daily = df_filtrado.groupby('TRAFO', group_keys=False).apply(lambda x: x.set_index('datahora').resample('D').max()).reset_index()

df_count = df_daily.groupby('TRAFO').count()
df_count = df_count.sort_values('datahora').tail(41).reset_index()

lista_trafos = df_count['TRAFO'].unique().tolist()
# lista_trafos
trafos_escolhidos = lista_trafos

df_filtrado = df_daily[df_daily['TRAFO'].isin(lista_trafos)]


In [ ]:
# # Criando gráfico da Potência Aparente ao longo do tempo, separada por TRAFO
# fig_aparente = px.line(df_filtrado, x='datahora', y='S', color='TRAFO',
#                        title='Potência Aparente ao Longo do Tempo por Transformador',
#                        labels={'S': 'Potência Aparente (kVA)', 'Dia': 'Data'})

# # Exibir ou salvar o gráfico em
# fig_aparente.show()
# fig_aparente.write_html("Demanda ao longo do tempo - IQR.html")

In [ ]:
data_inicio = '2018-01-01'
data_fim = '2024-12-31'
datas_completas = pd.date_range(start=data_inicio, end=data_fim, freq='D')

def preencher_com_7_linhas_anteriores(grupo):
    grupo = grupo.set_index('datahora')
    grupo = grupo.reindex(datas_completas)

    grupo = grupo.fillna(grupo.shift(7))  # 'shift(7)' pega o valor de 7 linhas antes

    grupo['TRAFO'] = grupo['TRAFO'].iloc[0]  # Reatribui o trafo
    return grupo.reset_index().rename(columns={'index': 'datahora'})

def preencher_com_1_linhas_anteriores(grupo):
    grupo = grupo.set_index('datahora')
    grupo = grupo.reindex(datas_completas)

    grupo = grupo.fillna(grupo.shift(1))  # 'shift(1)' pega o valor de 1 linha antes

    grupo['TRAFO'] = grupo['TRAFO'].iloc[0]
    return grupo.reset_index().rename(columns={'index': 'datahora'})

# Aplicar a função para cada transformador
df_preenchido = df_filtrado.groupby('TRAFO', group_keys=False).apply(preencher_com_7_linhas_anteriores)
df_preenchido = df_preenchido.groupby('TRAFO', group_keys=False).apply(preencher_com_7_linhas_anteriores)
df_preenchido = df_preenchido.groupby('TRAFO', group_keys=False).apply(preencher_com_7_linhas_anteriores)
df_preenchido = df_preenchido.groupby('TRAFO', group_keys=False).apply(preencher_com_7_linhas_anteriores)
df_preenchido = df_preenchido.groupby('TRAFO', group_keys=False).apply(preencher_com_7_linhas_anteriores)
df_preenchido = df_preenchido.groupby('TRAFO', group_keys=False).apply(preencher_com_7_linhas_anteriores)
df_preenchido = df_preenchido.groupby('TRAFO', group_keys=False).apply(preencher_com_1_linhas_anteriores)


In [ ]:
# # Criando gráfico da Potência Aparente ao longo do tempo, separada por TRAFO
# fig_aparente = px.line(df_preenchido, x='datahora', y='S', color='TRAFO',
#                        title='Potência Aparente ao Longo do Tempo por Transformador',
#                        labels={'S': 'Potência Aparente (kVA)', 'Dia': 'Data'})

# # Exibir ou salvar o gráfico em
# fig_aparente.show()

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.5 MB/s eta 0:00:00


#### Caso precise, ST

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
import random

# Fixando seeds para reprodutibilidade
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

def plotar_resultados(df_previsoes, y_test_inverso, y_pred, trafo, modelo):
    plt.figure(figsize=(14, 6))
    plt.plot(df_previsoes, y_test_inverso, label='Valores Reais', color='blue')
    plt.plot(df_previsoes, y_pred, label=f'Predictions {modelo} - Transformer {trafo}', linestyle='--', color='orange')
    plt.xlabel('Date')
    plt.ylabel('Apparent Power')
    plt.title(f'Predicted x Real Comparison - Transformer {trafo} ({modelo})')
    plt.legend()
    plt.show()

def treinar_e_prever_modelo(df_filtrado, trafos_escolhidos, modelo, janela, epochs=20, batch_size=32):
    resultados = []

    for trafo in trafos_escolhidos:
        df = df_filtrado[df_filtrado['TRAFO'] == trafo]
        df = df[['datahora', 'S']]
        df = df.set_index(['datahora'])#.resample('D').max()
        df.sort_index(inplace=True)
        # df['S'] = df['S'].interpolate(method='linear')

        dados = df[['S']].values
        scaler = MinMaxScaler(feature_range=(0, 1))
        dados_normalizados = scaler.fit_transform(dados)

        X, y = [], []
        for i in range(janela, len(dados_normalizados)):
            X.append(dados_normalizados[i-janela:i, 0])
            y.append(dados_normalizados[i, 0])

        X, y = np.array(X), np.array(y)
        X = np.reshape(X, (X.shape[0], X.shape[1], 1)) if modelo == 'LSTM' else X

        split = int(len(X) * 0.83)
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        if modelo == 'SVR':
            regressor = SVR(kernel='rbf', C=100, gamma=0.001, epsilon=0.01)

        elif modelo == 'GBR':
            regressor = GradientBoostingRegressor(n_estimators=100, random_state=42)
        elif modelo == 'LGBM':
            regressor = LGBMRegressor(n_estimators=100, random_state=42)
        elif modelo == 'XGB':
            regressor = XGBRegressor(n_estimators=100, random_state=42)
        elif modelo == 'CatBoost':
            regressor = CatBoostRegressor(n_estimators=100, random_state=42, verbose=0)
        elif modelo == 'RFR':
            regressor = RandomForestRegressor(n_estimators=100, max_depth=20, max_features='sqrt', n_jobs=-1, random_state=42)
        elif modelo == 'LSTM':
            regressor = Sequential()
            regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
            regressor.add(LSTM(units=50))
            regressor.add(Dense(1))
            regressor.compile(optimizer='adam', loss='mean_squared_error')
            regressor.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

        regressor.fit(X_train, y_train) if modelo != 'LSTM' else None

        y_pred_normalizado = regressor.predict(X_test)
        y_pred = scaler.inverse_transform(y_pred_normalizado.reshape(-1, 1))
        y_test_inverso = scaler.inverse_transform(y_test.reshape(-1, 1))

        mse = mean_squared_error(y_test_inverso, y_pred)
        rmse = sqrt(mse)
        mae = mean_absolute_error(y_test_inverso, y_pred)
        r2 = r2_score(y_test_inverso, y_pred)

        df_previsoes = df.index[split + janela:]
        plotar_resultados(df_previsoes, y_test_inverso, y_pred, trafo, modelo)

        resultados.append({
            'Trafo': trafo,
            'Modelo': modelo,
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2
        })

    return pd.DataFrame(resultados)

# Chamando para SVR, RFR e LSTM
# resultados_svr = treinar_e_prever_modelo(df_preenchido, trafos_escolhidos, modelo='SVR', janela=365)
# resultados_rfr = treinar_e_prever_modelo(df_preenchido, trafos_escolhidos, modelo='RFR', janela=365)
resultados_gbr = treinar_e_prever_modelo(df_preenchido, trafos_escolhidos, modelo='GBR', janela=365)
print(resultados_gbr)
resultados_lgbm = treinar_e_prever_modelo(df_preenchido, trafos_escolhidos, modelo='LGBM', janela=365)
print(resultados_lgbm)
resultados_xgb = treinar_e_prever_modelo(df_preenchido, trafos_escolhidos, modelo='XGB', janela=365)
print(resultados_xgb)
resultados_catboost = treinar_e_prever_modelo(df_preenchido, trafos_escolhidos, modelo='CatBoost', janela=365)
print(resultados_catboost)
# resultados_lstm = treinar_e_prever_modelo(df_preenchido, trafos_escolhidos, modelo='LSTM', janela=365, epochs=20, batch_size=32)

# resultados_svr.to_csv('SVR_S.csv', sep=';')
# resultados_rfr.to_csv('RFR_S.csv', sep=';')
# resultados_lstm.to_csv('LSTM_S.csv', sep=';')

# print("Resultados SVR:")
# print(resultados_svr)
# print("\nResultados RFR:")
# print(resultados_rfr)
# print("\nResultados LSTM:")
# print(resultados_lstm)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
import random

# Fixando seeds para reprodutibilidade
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

def plotar_resultados(df_previsoes, y_test_inverso, y_pred, trafo, modelo):
    plt.figure(figsize=(14, 6))
    plt.plot(df_previsoes, y_test_inverso, label='Valores Reais', color='blue')
    plt.plot(df_previsoes, y_pred, label=f'Predictions {modelo} - Transformer {trafo}', linestyle='--', color='orange')
    plt.xlabel('Date')
    plt.ylabel('Apparent Power')
    plt.title(f'Predicted x Real Comparison - Transformer {trafo} ({modelo})')
    plt.legend()
    plt.show()

def treinar_e_prever_modelo(df_filtrado, trafos_escolhidos, modelo, janela, epochs=20, batch_size=32):
    resultados = []

    for trafo in trafos_escolhidos:
        df = df_filtrado[df_filtrado['TRAFO'] == trafo]
        df = df[['datahora', 'S']]
        df = df.set_index(['datahora'])#.resample('D').max()
        df.sort_index(inplace=True)
        # df['S'] = df['S'].interpolate(method='linear')

        dados = df[['S']].values
        scaler = MinMaxScaler(feature_range=(0, 1))
        dados_normalizados = scaler.fit_transform(dados)

        X, y = [], []
        for i in range(janela, len(dados_normalizados)):
            X.append(dados_normalizados[i-janela:i, 0])
            y.append(dados_normalizados[i, 0])

        X, y = np.array(X), np.array(y)
        X = np.reshape(X, (X.shape[0], X.shape[1], 1)) if modelo == 'LSTM' else X

        split = int(len(X) * 0.83)
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        if modelo == 'SVR':
            regressor = SVR(kernel='rbf', C=100, gamma=0.001, epsilon=0.01)

        elif modelo == 'GBR':
            regressor = GradientBoostingRegressor(n_estimators=100, random_state=42)
        elif modelo == 'LGBM':
            regressor = LGBMRegressor(n_estimators=100, random_state=42)
        elif modelo == 'XGB':
            regressor = XGBRegressor(n_estimators=100, random_state=42)
        elif modelo == 'CatBoost':
            regressor = CatBoostRegressor(n_estimators=100, random_state=42, verbose=0)
        elif modelo == 'RFR':
            regressor = RandomForestRegressor(n_estimators=100, max_depth=20, max_features='sqrt', n_jobs=-1, random_state=42)
        elif modelo == 'LSTM':
            regressor = Sequential()
            regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
            regressor.add(LSTM(units=50))
            regressor.add(Dense(1))
            regressor.compile(optimizer='adam', loss='mean_squared_error')
            regressor.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

        regressor.fit(X_train, y_train) if modelo != 'LSTM' else None

        y_pred_normalizado = regressor.predict(X_test)
        y_pred = scaler.inverse_transform(y_pred_normalizado.reshape(-1, 1))
        y_test_inverso = scaler.inverse_transform(y_test.reshape(-1, 1))

        mse = mean_squared_error(y_test_inverso, y_pred)
        rmse = sqrt(mse)
        mae = mean_absolute_error(y_test_inverso, y_pred)
        r2 = r2_score(y_test_inverso, y_pred)

        df_previsoes = df.index[split + janela:]
        plotar_resultados(df_previsoes, y_test_inverso, y_pred, trafo, modelo)

        resultados.append({
            'Trafo': trafo,
            'Modelo': modelo,
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2
        })

    return pd.DataFrame(resultados)

# Chamando para SVR, RFR e LSTM
resultados_svr = treinar_e_prever_modelo(df_preenchido, trafos_escolhidos, modelo='SVR', janela=365)
print(resultados_svr)
resultados_rfr = treinar_e_prever_modelo(df_preenchido, trafos_escolhidos, modelo='RFR', janela=365)
print(resultados_rfr)
resultados_lstm = treinar_e_prever_modelo(df_preenchido, trafos_escolhidos, modelo='LSTM', janela=365, epochs=20, batch_size=32)
print(resultados_lstm)
# resultados_svr.to_csv('SVR_S.csv', sep=';')
# resultados_rfr.to_csv('RFR_S.csv', sep=';')
# resultados_lstm.to_csv('LSTM_S.csv', sep=';')

# print("Resultados SVR:")
# print(resultados_svr)
# print("\nResultados RFR:")
# print(resultados_rfr)
# print("\nResultados LSTM:")
# print(resultados_lstm)


In [ ]:
resultados_svr.to_csv('SVR_S.csv', sep=';')
resultados_rfr.to_csv('RFR_S.csv', sep=';')
# resultados_lstm.to_csv('LSTM_S.csv', sep=';')
resultados_gbr.to_csv('GBR_S.csv', sep=';')
resultados_xgb.to_csv('XGB_S.csv', sep=';')
resultados_lgbm.to_csv('LGBM_S.csv', sep=';')
resultados_catboost.to_csv('catboost_S.csv', sep=';')